# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
from sklearn.model_selection import train_test_split
X = adult_dt.drop(columns='income')
Y = adult_dt['income']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random_state in train_test_split sets a seed for reproducible shuffling. Fixing it ensures the same data split across runs, making results consistent and comparable, which is essential for reproducibility in machine learning.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [3]:
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

pipe_num_simple = Pipeline([
    ('knn', KNNImputer(n_neighbors=3)),
    ('scaler', RobustScaler())
])

pipe_cat_simple = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'))
])

transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', pipe_num_simple, ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']),
        ('cat_transforms', pipe_cat_simple, ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']), 
    ], remainder='passthrough'
)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [5]:
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline(
    [
        ('preprocessing', transformer), 
        ('classifier', RandomForestClassifier(max_depth=2, random_state=42))
    ]
)

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [6]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']
results_dict = cross_validate(pipe, X_train, Y_train, cv=5, scoring = scoring, return_train_score = True)
results_dict

{'fit_time': array([0.72110915, 0.69114161, 0.70511413, 0.78988647, 0.85618067]),
 'score_time': array([0.15055728, 0.11170101, 0.11269617, 0.12267256, 0.11174345]),
 'test_neg_log_loss': array([-0.43998206, -0.43785599, -0.43769782, -0.43751068, -0.43834932]),
 'train_neg_log_loss': array([-0.43886263, -0.43728559, -0.43760816, -0.43779162, -0.43865275]),
 'test_roc_auc': array([0.87707219, 0.88888348, 0.8902713 , 0.8877241 , 0.8876574 ]),
 'train_roc_auc': array([0.8785876 , 0.88919751, 0.88841012, 0.88878171, 0.88837693]),
 'test_accuracy': array([0.76332529, 0.77604738, 0.77029399, 0.77204914, 0.77665643]),
 'train_accuracy': array([0.76317666, 0.77502331, 0.77415817, 0.77289679, 0.77251289]),
 'test_balanced_accuracy': array([0.51220615, 0.53842676, 0.52624434, 0.52986425, 0.53936652]),
 'train_balanced_accuracy': array([0.51164895, 0.5362316 , 0.53452733, 0.53184982, 0.53105833])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [7]:
pd.DataFrame(results_dict).sort_values('test_neg_log_loss')

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
0,0.721109,0.150557,-0.439982,-0.438863,0.877072,0.878588,0.763325,0.763177,0.512206,0.511649
4,0.856181,0.111743,-0.438349,-0.438653,0.887657,0.888377,0.776656,0.772513,0.539367,0.531058
1,0.691142,0.111701,-0.437856,-0.437286,0.888883,0.889198,0.776047,0.775023,0.538427,0.536232
2,0.705114,0.112696,-0.437698,-0.437608,0.890271,0.888410,0.770294,0.774158,0.526244,0.534527
3,0.789886,0.122673,-0.437511,-0.437792,0.887724,0.888782,0.772049,0.772897,0.529864,0.531850


Calculate the mean of each metric. 

In [8]:
pd.DataFrame(results_dict).mean()

fit_time                   0.752686
score_time                 0.121874
test_neg_log_loss         -0.438279
train_neg_log_loss        -0.438040
test_roc_auc               0.886322
train_roc_auc              0.886671
test_accuracy              0.771674
train_accuracy             0.771554
test_balanced_accuracy     0.529222
train_balanced_accuracy    0.529063
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [9]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

pipe.fit(X_train, Y_train)

Y_pred = pipe.predict(X_test)
Y_pred_proba = pipe.predict_proba(X_test)

roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])
neg_log_loss = -log_loss(Y_test, Y_pred_proba)
accuracy = accuracy_score(Y_test, Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)

test_results_dict = {
    'roc_auc': round(roc_auc, 3),
    'neg_log_loss': round(neg_log_loss, 3),
    'accuracy': round(accuracy, 3),
    'balanced_accuracy': round(balanced_accuracy, 3)
}

print(test_results_dict)

{'roc_auc': 0.888, 'neg_log_loss': -0.434, 'accuracy': 0.78, 'balanced_accuracy': 0.535}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding the target variable income into binary (0 and 1) is convenient because it ensures compatibility with binary classification models and evaluation metrics, simplifying the entire workflow.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.